In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os

from keras.layers import Concatenate, Input, Dense, Embedding, Flatten, Dropout, BatchNormalization, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import  Adam
import keras.backend as k

# import matplotlib.pyplot as plt
# %matplotlib inline
# from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
# pd.options.display.precision = 15
from category_encoders.cat_boost import CatBoostEncoder

# import lightgbm as lgb
# import xgboost as xgb
# import time
# import datetime
# from catboost import CatBoostRegressor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
# from sklearn import metrics
# from sklearn import linear_model
import gc
import pickle
# import seaborn as sns
# import warnings
# warnings.filterwarnings("ignore")

# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt

# import networkx as nx
# import matplotlib.pyplot as plt

gc.collect()
# alt.renderers.enable('notebook')

Using TensorFlow backend.
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

0

In [2]:
main_path = r'd:\Documents\Private\Kaggle\IEEEFraud'

In [15]:
import sys
sys.path.append(main_path)
from BayDS import *

In [4]:
experiment_name = '31.08'
main_learning_folder = main_path+'/Snapshots/'+experiment_name


In [ ]:
train = pd.read_pickle(f'{main_learning_folder}/train.pkl')
test = pd.read_pickle(f'{main_learning_folder}/test.pkl')


In [ ]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT'], axis=1)
y = train.sort_values('TransactionDT')['isFraud'].astype(np.uint8)
test = test.sort_values('TransactionDT').drop(['TransactionDT'], axis=1)

X.drop('Date', axis=1, inplace=True)
X=X.astype(np.float32)
test.drop('Date', axis=1, inplace=True)
test=test.astype(np.float32)
del train

In [ ]:
gc.collect()

In [ ]:
merg = pd.read_pickle(f'{main_learning_folder}/catboostencoder.pkl')

In [5]:
cat_anya = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15',
       'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33',
       'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo',
       'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin',
       'R_emaildomain_suffix', 'device_name', 'device_version', 'OS_id_30',
       'version_id_30', 'browser_id_31', 'version_id_31', 'screen_width',
       'screen_height', 'uid', 'uid2', 'uid3']

In [6]:
cat_vasya = ["id_13", "id_14", "id_17", "id_18", "id_19", "id_20", "id_21", "id_22", "id_23", 
 "id_24", "id_25", "id_26", "id_27", "id_30", "id_31", "id_33", "DeviceType", 
 "DeviceInfo", "P_emaildomain", "R_emaildomain", "card1", "card2", "card3", 
 "card5", "addr1", "addr2", "P_emaildomain_bin", "P_emaildomain_suffix", 
 "R_emaildomain_bin", "R_emaildomain_suffix", "M1", "M2", "M3", "M4", "M5", 
 "M6", "M7", "M8", "M9", "card4", "card6", "ProductCD", "id_12", "id_15", 
 "id_16", "id_28", "id_29", "id_32", "id_34", "id_35", "id_36", "id_37", "id_38"]

In [7]:
cat_features = list(set(cat_vasya) | set(cat_anya))

In [8]:
num_features = list(set(X.columns) - set(cat_features))

NameError: name 'X' is not defined

In [ ]:
num_features

In [ ]:
# for col in cat_features:
#     alld = pd.concat([X[col],test[col]])
#     num_values = len(alld.unique()) 
#     if num_values > 20:
#         print(col, '- Big categroical')
#         X[col] = merg[col].loc[X.index]
#         test[col]= merg[col].loc[test.index]
#     elif num_values>2:
#         print(col,'- one-hot')
#         temp = pd.get_dummies(alld,prefix=f'{col}_')
#         X=pd.concat([X,temp.loc[X.index]],axis=1)
#         X.drop([col],axis=1,inplace=True)
#         test=pd.concat([test,temp.loc[test.index]],axis=1)
#         test.drop([col],axis=1,inplace=True)
        
# X.to_pickle(f'{main_learning_folder}/X_After_encoding.pkl')
# test.to_pickle(f'{main_learning_folder}/test_after_encoding.pkl')

X = pd.read_pickle(f'{main_learning_folder}/X_After_encoding.pkl')
test = pd.read_pickle(f'{main_learning_folder}/test_after_encoding.pkl')

    

In [ ]:
del merg
gc.collect()

In [ ]:
for column in num_features:
    print(column)
    col = pd.concat([X[column],test[column]])
    filtered = col[~col.isin([np.inf, -np.inf])].dropna()
    vmax = filtered.max()
    vmin = filtered.min()
    vmean = filtered.mean()
    col = col.replace({
        np.inf:vmax,
        -np.inf:vmin
       }).fillna(vmean)
    if vmax > 100 and vmin >= 0:
        col = np.log1p(col)
    
    scaler = StandardScaler()
    col.values[...] = scaler.fit_transform(col.values.reshape(-1,1)).flatten()
    
    X[column] = col[X.index]
    test[column] = col[test.index]
    
    

In [ ]:
X.to_pickle(f'{main_learning_folder}/X_encoded_scaled.pkl')
y.to_pickle(f'{main_learning_folder}/y.pkl')
test.to_pickle(f'{main_learning_folder}/test_encoded_scaled.pkl')

In [9]:
#start here
y = pd.read_pickle(f'{main_learning_folder}/y.pkl')
X = pd.read_pickle(f'{main_learning_folder}/X_encoded_scaled.pkl')
test = pd.read_pickle(f'{main_learning_folder}/test_encoded_scaled.pkl')
gc.collect()

27

In [10]:
# Setting model_folder
model_name = 'keras-2'
model_folder = f'{main_learning_folder}/{model_name}'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [11]:
n_fold = 5
# folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=n_fold)
# folds = GroupKFold(n_splits=5)
# groups = pd.read_pickle('./groups.pkl').values

In [ ]:
y.value_counts()

In [12]:
def NNModel_maker():
    k.clear_session()
    
#     categorical_inputs = []
#     for cat in categorical:
#         categorical_inputs.append(Input(shape=[1], name=cat))

#     categorical_embeddings = []
#     for i, cat in enumerate(categorical):
#         categorical_embeddings.append(
#             Embedding(category_counts[cat], int(np.log1p(category_counts[cat]) + 1), name = cat + \
#                       "_embed")(categorical_inputs[i]))

#     categorical_logits = Concatenate(name = "categorical_conc")([Flatten()(SpatialDropout1D(.1)(cat_emb)) for cat_emb in categorical_embeddings])
# 
    numerical_inputs = Input(shape=[X.shape[1]], name = 'all')
    numerical_logits = Dropout(.3)(numerical_inputs)
  
    x = numerical_logits

    x = Dense(200, activation = 'relu')(x)
    x = Dropout(.3)(x)
    x = Dense(100, activation = 'relu')(x)
    x = Dropout(.3)(x)
    out = Dense(1, activation = 'sigmoid')(x)    

    model = Model(inputs= [numerical_inputs],outputs=out)
    loss = "binary_crossentropy"
    model.compile(optimizer=Adam(lr = 0.0003), loss = loss)
    return model


params = {
    'batch_size': 8000,
    'epochs': 30,
    'verbose': True,
         }
train_options = {
    "model_type":'keras',
    "params": params,
    "eval_metric":'auc',
    'averaging': 'usual',
    'use_groups': False,
    'fold_name': folds.__class__.__name__,
    'n_splits': n_fold
   
}

In [13]:
with open(f'{model_folder}/training_params.json', 'w') as f:
    q = json.dumps(train_options,indent=2)
    f.write(q)
# NNModel.save(f'{model_folder}/keras.mdl')

In [ ]:
gc.collect()
result_dict_keras = train_model_classification(model=NNModel_maker, 
                                             X=X,
                                             X_test=test,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             eval_metric=train_options['eval_metric'],
                                             plot_feature_importance=True,
                                             averaging=train_options['averaging'],
                                             groups=None)

Fold 1 started at Sun Sep  1 20:22:23 2019
Train on 472432 samples, validate on 118108 samples
Epoch 1/30
472432/472432 [==============================] - 23s 49us/step - loss: 0.2062 - val_loss: 0.1043
Epoch 2/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.1286 - val_loss: 0.1084
Epoch 3/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.1232 - val_loss: 0.1068
Epoch 4/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.1184 - val_loss: 0.1056
Epoch 5/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.1155 - val_loss: 0.1016
Epoch 6/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.1125 - val_loss: 0.0986
Epoch 7/30
472432/472432 [==============================] - 21s 44us/step - loss: 0.1105 - val_loss: 0.0992
Epoch 8/30
472432/472432 [==============================] - 22s 46us/step - loss: 0.1080 - val_loss: 0.0974
Epoch 9/30
472432/472432 [===============

472432/472432 [==============================] - 21s 45us/step - loss: 0.0927 - val_loss: 0.0925
Epoch 13/30
472432/472432 [==============================] - 21s 44us/step - loss: 0.0909 - val_loss: 0.0912
Epoch 14/30
472432/472432 [==============================] - 21s 44us/step - loss: 0.0897 - val_loss: 0.0900
Epoch 15/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.0887 - val_loss: 0.0885
Epoch 16/30
472432/472432 [==============================] - 22s 47us/step - loss: 0.0876 - val_loss: 0.0881
Epoch 17/30
472432/472432 [==============================] - 21s 45us/step - loss: 0.0870 - val_loss: 0.0869
Epoch 18/30
472432/472432 [==============================] - 21s 44us/step - loss: 0.0855 - val_loss: 0.0862
Epoch 19/30
472432/472432 [==============================] - 20s 43us/step - loss: 0.0851 - val_loss: 0.0855
Epoch 20/30
472432/472432 [==============================] - 20s 43us/step - loss: 0.0838 - val_loss: 0.0842
Epoch 21/30
472432/472432 [====

In [ ]:
sub = pd.read_csv(f'../../data/sample_submission.csv')

In [ ]:
sub['isFraud'] = result_dict_keras['prediction']
sub.to_csv('ieee_nn.csv', index=False)

In [ ]:
result_dict_keras['prediction'].shape

In [ ]:
test.shape

In [ ]:
import pickle
with open(f'{model_folder}/results_dict.pkl', 'wb') as f:
#     q = json.dumps(result_dict_lgb,indent=2)
    pickle.dump(result_dict_keras,f)
#     f.write(q)